<a href="https://colab.research.google.com/github/maipu/Variable-Analysis-Ozone/blob/master/00_VAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variable Analysis: Ozone.

Santiago de Chile.  
Estaciones SINCA.

|       | Verano inicial | Verano Final |
| ----- | -------------- | ------------ |
|CONDES |      2004      |  2013     |
|INDEP  |      2010      |  2017     |
|POH	|      2009      |  2016     |
|CERRI  |      2004      |  2013     |

# Detectar COLAB

Intenta detectar cavernicolamente si está en COLAB.

In [1]:
import sys
dir1 = '/usr/local/lib/'
dir2 = '/root/.local/share/jupyter/'
__a, __b, __c = sys.argv
COLAB = False
if __a[:len(dir1)] == dir1 and __c[:len(dir2)] == dir2:
    COLAB = True

# Clonar el repo de GitHub

Esto debería ser sólo para colab.
* Clona el repositorio para tener acceso a los datasets y otros archivos.
* Crea vínculos simbólicos para que el código de jupyter-notebook sea compatible con colab.

In [2]:
# Import de GitHub

if COLAB == True:
    import os

    !git clone https://github.com/maipu/Variable-Analysis-Ozone.git
    print("\nCreando vínculos simbólicos ...")
    !ln -s -f -v Variable-Analysis-Ozone/datasets
    !ln -s -f -v Variable-Analysis-Ozone/search_outputs
    !ln -s -f -v Variable-Analysis-Ozone/stored_files
    !ln -s -f -v Variable-Analysis-Ozone/utils.py
    print("LISTO")
    !rm -r -f ./sample_data

Cloning into 'Variable-Analysis-Ozone'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 177 (delta 7), reused 24 (delta 4), pack-reused 144
Receiving objects: 100% (177/177), 55.83 MiB | 20.25 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Checking out files: 100% (53/53), done.

Creando vínculos simbólicos ...
'./datasets' -> 'Variable-Analysis-Ozone/datasets'
'./search_outputs' -> 'Variable-Analysis-Ozone/search_outputs'
'./stored_files' -> 'Variable-Analysis-Ozone/stored_files'
'./utils.py' -> 'Variable-Analysis-Ozone/utils.py'
LISTO


In [3]:
# Función necesaria siempre que se quiera mostrar un gráfico en una celda.
 
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

# Ejecutar todo en jupyter-notebook local

Este botón es para quienes no se manejan mucho con jupyter-notebooks. Sirve de ayuda para iniciar todo el proceso.

El code_toggle no funciona en colab.



In [4]:
from IPython.display import HTML, Javascript, display
from ipywidgets import widgets

def start(ev):
    display(Javascript('IPython.notebook.execute_cell_range(0+1, 6+10)'))
print("If the button doesn't appear, click here and press run.")


button = widgets.Button(description="Run THIS FIRST")
if COLAB == True:
    button.description = "No en COLAB =)"
    button.disabled = True
button.on_click(start)
display(button)

If the button doesn't appear, click here and press run.


Button(description='No en COLAB =)', disabled=True, style=ButtonStyle())

In [5]:
if COLAB == False:
    def code_toggle():
        display(HTML('''<script>
        function code_toggle() {
            if (code_shown){
            $('div.input').hide('500');
            $('#toggleButton').val('Show Code')
            } else {
            $('div.input').show('500');
            $('#toggleButton').val('Hide Code')
            }
            code_shown = !code_shown
        }
        
        $( document ).ready(function(){
            code_shown=false;
            $('div.input').hide()
        });
        </script>
        <form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>'''))


    code_toggle()

# Imports

In [6]:
from IPython.display import Markdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import plotly.plotly as py
from plotly.offline import init_notebook_mode, enable_mpl_offline, iplot_mpl, iplot

import cufflinks as cf
from sklearn.feature_selection import mutual_info_regression

import utils
from utils import gpr_invierno, gpr_verano

cf.go_offline(connected=True)
if COLAB == False:
    init_notebook_mode(connected=True)
    enable_mpl_offline()

# 1) Seleccionar Dataset

Carga los datos entre 01 Enero de 1997 y 12 Abril de 2018.

Se hacen los grupos por estación:  
Invierno: 01 de Mayo al 31 de Agosto.  
Verano: 01 de Noviembre al 31 de Marzo.

**Nota:** Para casi todo (gráficos y boxplots) se utiliza data diaria. Para el cálculo de las correlaciones se utiliza la data horaria.

In [7]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}

from ipywidgets import widgets, interactive, interact
from IPython.display import Javascript, display



ozone = None
dataset_name = 'DATASET_NONE!'
dataset_sigla = 'SIGLA_NONE!'
STATIONS = ['Las Condes','Independencia', "Parque O'Higgins", "Cerrillos"]
gpi_hourly = None
gpi = None
gpv_hourly = None
gpv = None
SINCE = 1997
UNTIL = 2017

def hacer_grupos(ozone, SINCE, UNTIL):
    global gpi_hourly, gpi, gpv_hourly, gpv

    ozone['WD-sin'] = ozone['WD'].apply(lambda x: np.sin(x*np.pi/180))
    ozone['WD-cos'] = ozone['WD'].apply(lambda x: np.cos(x*np.pi/180))

    invierno = utils.df_invierno(ozone, SINCE, UNTIL) #selección de registros pertenecientes a invierno
    verano = utils.df_verano(ozone, SINCE, UNTIL)

    gpi_hourly = invierno.groupby(by=gpr_invierno) #agrupa los registros de invierno por año.
    gpi = invierno.groupby(pd.Grouper(freq='D'))
    gpi = gpi.aggregate(np.max)
    gpi = gpi.groupby(by=gpr_invierno)
    print("Grupos de Invierno actualizados.")

    gpv_hourly = verano.groupby(by=gpr_verano)
    gpv = verano.groupby(pd.Grouper(freq='D'))
    gpv = gpv.aggregate(np.max)
    gpv = gpv.groupby(by=gpr_verano)
    print("Grupos de Verano actualizados.")

def select_dataset(dataset='Las Condes'):
    global ozone, dataset_name, dataset_sigla
    if dataset == 'Las Condes':
        ozone = pd.read_hdf("datasets/dump-Las_Condes_2018-04-12_230000.h5","table")
        #dataset_name = dataset
        dataset_sigla = "CONDES"
    elif dataset == 'Independencia':
        ozone = pd.read_hdf("datasets/dump-Independencia_2018-04-12_230000.h5","table")
        #dataset_name = dataset
        dataset_sigla = "INDEP"
    elif dataset == "Parque O'Higgins":
        ozone = pd.read_hdf("datasets/dump-Parque_OHiggins_2019-06-14_230000.h5","table")
        #dataset_name = dataset
        dataset_sigla = "POH"
    elif dataset == "Cerrillos":
        ozone = pd.read_hdf("datasets/dump-Cerrillos_2017-01-12_110000.h5","table")
        dataset_sigla = "CERRI"
    dataset_name = dataset
    print("\nDataset cambiado a: {}".format(dataset_sigla))
    hacer_grupos(ozone, 1997, 2017)
    print("LISTO")
    #display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells() )'))
    
    
dataset_wg = widgets.Dropdown(options=STATIONS, value='Las Condes')


interact(select_dataset, dataset=STATIONS)
#print(dataset_name)
print()


interactive(children=(Dropdown(description='dataset', options=('Las Condes', 'Independencia', "Parque O'Higgin…

# 2) Visualización de datos

In [8]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}
display(Markdown('## Series de tiempo - Estación: {}.'.format(dataset_name)))
configure_plotly_browser_state()

from ipywidgets import widgets, interactive

def show_group(season='Verano', year_i=1996, year_f=2018, month='All', predictor='O3', hourly=False):
    if month == None:
        return
    if hourly == True:
        tipo = 'Hourly'
    else:
        tipo = 'Daily'
    title = "{} - Time Series for {} - {} Max.".format(dataset_name, predictor, tipo)
    
    if season == 'Invierno':
        if hourly:
            group = gpi_hourly
        else:
            group = gpi
    elif season == 'Verano':
        if hourly:
            group = gpv_hourly
        else:
            group = gpv
        
    
    if year_i == 'All':
        months_wg.disabled = True
        df = group.filter(lambda x: True)
        '''
        if year_f == 'All':
            year_f=2017
            
        df = ozone[ (ozone.index.year>=1996) & (ozone.index.year<=int(year_f)+1)]
        o3 =  df['O3'].ffill()
        o3 =  o3[o3.first_valid_index():o3.last_valid_index()]
        print(len(o3.dropna())/len(o3))
        '''
    else:
        months_wg.disabled = False
        #df = group.get_group(year)
        if year_f == "All":
            year_f=2017
        df = group.filter(lambda x: True)["%s-11-01"%(year_i):"%s-03-31"%(year_f+1)]
        if month != 'All' and year_i != year_f:
            group = df.groupby(df.index.month)
            df = group.get_group(utils.month_number(month))
        elif month != 'All':
            group = df.groupby(df.index.month)
            df = group.get_group(utils.month_number(month))
        
    if predictor == 'All':
        output = df
    else:
        output = df[[predictor]]#.ffill()
    
    #display(output.iplot())
    output.iplot(title=title)
    display( output.describe() )
    print("Quantile 0.9")
    display(output.quantile(0.9, axis=0) )
    print("\n")
    print('Info para datos faltantes. (Sólo para Verano)')
    print('Features consideradas: CO, PM10, PM25, NO, NOX, WD, RH, TEMP, WS, UVA, UVB, O3')
    output2 = output[ (output.index.month>=11) |  (output.index.month<=3) ]
    DIAS = len(output2)
    print("DIAS:", DIAS)
    FEATURES = ['CO', 'PM10', 'PM25', 'NO', 'NOX', 'WD', 'RH', 'TEMP', 'WS', 'UVA', 'UVB', 'O3']
    F_LEN = len(FEATURES)
    ALLCOUNT = F_LEN * DIAS
    print("[MC] MAX COUNT (FEATURES * DIAS):", F_LEN * DIAS )
    COUNT = df[FEATURES].count().sum()
    print( "[FC] REAL FEATURES COUNT:", COUNT )
    MISSING = (1 - COUNT/ALLCOUNT)*100
    print("Missing = 1 - MC/FC")
    print("Missing data: {0:.2f}%".format(MISSING))
    #display( output )
    

seasons_wg = widgets.Dropdown(options=['Invierno','Verano'],
                           value='Verano')

s_months={ 'Invierno':[], 'Verano':[] }
s_months['Invierno'] += ['All'] + list(map(utils.month_name, utils.inv_range))
s_months['Verano'] += ['All'] + list(map(utils.month_name, utils.ver_range))
months_wg = widgets.Dropdown(options=s_months[seasons_wg.value],
                          value='All')
def on_update_season_widget( *kwargs):
    months_wg.options = s_months[seasons_wg.value]

seasons_wg.observe(on_update_season_widget,'value')

years = ['All'] + list(range(SINCE,UNTIL+1))
years_inicio = ['All'] + list(range(SINCE,UNTIL+1))
years_fin = ['All'] + list(range(SINCE,UNTIL+1))
predictors = ['All']+ list(ozone.columns)


w = interactive(show_group, Hourly=False, season=seasons_wg,
                year_i=years_inicio, year_f=years_fin,
                #year=years,
                month=months_wg, predictor=predictors
               )

wgs = w.children[:-1]
output = w.children[-1]
hbox = widgets.HBox(wgs)
wgs[-1].indent = False
vbox = widgets.VBox([hbox, output])

display(vbox)
w.update() #to show the plot the first time

## Series de tiempo - Estación: Las Condes.

# 3) Boxplot by season

In [9]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}
display(Markdown('## Boxsplot - Station: {}.'.format(dataset_name)))
configure_plotly_browser_state()

from ipywidgets import widgets, interactive, interact

def boxplot_by_month(season='Verano', year=2017, predictor='O3'):
    if season == 'Invierno':
        group = gpi
        ordered = utils.inv_range
        title = "{} {}".format(season, str(year))
    else:
        group = gpv
        ordered = utils.ver_range
        title = "{} - {} {}/{}  - Daily Max.".format(dataset_name,season, str(year),str(year+1))
    gg = group.get_group(year)[[predictor]]
    gg = gg.groupby(gg.index.month, sort=False)
    hh = pd.DataFrame()
    for name, gp in gg:
        if name in ordered:
            hh = pd.concat( [ hh, gp.reset_index()[predictor] ], ignore_index=True, axis=1)
    hh.columns = map(utils.month_name, ordered)
    
    boxplot = hh.iplot(kind='box', boxpoints = 'outliers', title=title )
    boxplot #for the iplot it's not necessary to call display()
    
    

seasons = ['Invierno', 'Verano']
years = range(SINCE,UNTIL+1)
predictors = list(ozone.columns)

w = interactive(boxplot_by_month, season=seasons, year=years, predictor=predictors)
wgs = w.children[:-1]
output = w.children[-1]
hbox = widgets.HBox(wgs)
vbox = widgets.VBox([hbox, output])

display(vbox)
w.update() #to show the plot the first time

## Boxsplot - Station: Las Condes.

# 4) Correlación y MI con O3 by season

Busca el lag con la máxima correlación o Mutual Information (MI) entre el Ozono y algún predictor.

## Procedimiento:

Para cada medición de Ozono en un tiempo $t$, se calcula su correlación con el predictor $p$ en el tiempo $t_{-x}$ donde $x$ va desde 0 hasta el lag máximo seleccionado. Luego para cada lag $x$ los resultados son promediados y graficados.

$t$ siempre corresponderá a cierta hora del día, por defecto 13:00 (1PM), es decir, lo que se busca determinar es la correlación de los predictores con el Ozono siempre a una misma hora del día, se escogío la 1PM ya que es cuando generalmente ocurren los máximos diarios de Ozono. Se decidió utilizar esta restricción ya que si se utiliza cada medición de ozono de horas correlativas, los periodos de noche afectan directamente en la correlación del Ozono.

Ejemplo para 13H con 3 lags máximos:

* Se verá la correlación entre $Ozono_{2004-12-20-13:00}$ con $p_{2004-12-20-13:00}$ para $x=0$.

* Para $x=1$ será la correlación entre $Ozono_{2004-12-20-13:00}$ con $p_{2004-12-20-12:00}$

* Para $x=2$ será la correlación entre $Ozono_{2004-12-20-13:00}$ con $p_{2004-12-20-11:00}$

* Para $x=1$ será la correlación entre $Ozono_{2004-12-20-13:00}$ con $p_{2004-12-20-10:00}$

Para el siguiente día se repite el proceso hasta llegar a la última fecha y se promedian todas las correlaciones para $x=0$, $x=1$, $x=2$ y $x=3$.

Para la MI el procedimiento es similar.

In [10]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}
display(Markdown('## Correlación y MI con O3 - Station: {}.'.format(dataset_name)))
display(Markdown('* Initial Season de 0 es lo mismo que 1997'))
display(Markdown('* Final Season de 0 es lo mismo que 2017'))
display(Markdown('* Seleccionar MI para calcular Mutial Information en lugar de la correlación  \n\n'))
configure_plotly_browser_state()

from ipywidgets import widgets, interactive, interact, interact_manual, Layout
#import plotly.graph_objs as go    
#yrange = go.Layout(
#        yaxis=dict(
#            range=[-1, 1]
#        )
#    )

progress = widgets.IntProgress(
       value=0,
       min=0,
       max=100,
       step=1,
       description='Loading:',
       bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
       orientation='horizontal'
)
progress.layout.visibility='hidden'

def button_on_click(b):
    b.disabled = True
    w.update()
    b.disabled = False


GG = None
def plot_corr():
    season = seasons.value
    predictor = predictors.value
    maxhour = maxhours.value #LAGS
    filter_hour = hour.value
    if iseason.value == 0:
        INITIAL_YEAR = 1997
    else:
        INITIAL_YEAR = iseason.value
    
    if fseason.value == 0:
        LAST_YEAR = 2017 + 1
    else:
        LAST_YEAR = fseason.value + 1
    
    

    
    dfo3 = ozone[ (ozone.index.year >= INITIAL_YEAR) & (ozone.index.year <=LAST_YEAR)]["O3"]
    
    dfo3 = dfo3[ (dfo3.index.month>=11) | (dfo3.index.month<=3) ]
    gp = dfo3.groupby(pd.Grouper(freq='D'))
    
    
    def hola(x):
        if x.dropna().empty:
            return np.nan
        else:
            try:
                #print(x.idxmax())
                return x.idxmax().hour
            except:
                print(x)
                print(x.idxmax())
                return x.idxmax().hour
            
    print("yolanda")
    dfo3 = gp.agg(hola)
    print(dfo3.mode())
    
    
    
    
    df = ozone[ (ozone.index.year >= INITIAL_YEAR) & (ozone.index.year <= LAST_YEAR)][[predictor, "O3"]]
    
    if predictor == 'O3':
        predictor = 'O3_2'
        df.columns = ["O3_2", "O3"]
        
    first_in_predictor = df[predictor].first_valid_index()
    if first_in_predictor == None:
        print("maximo", np.nan, np.nan)
        print("minimo", np.nan, np.nan)
        return
    
    
    
    i = max( first_in_predictor, df['O3'].first_valid_index())
    f = df['O3'].last_valid_index()
    df = df[i:f]
    df = df.asfreq(freq='H')
    
    
    #if season == 'Invierno':
    #    df = ozone[ (ozone.index.year > 1997) & (ozone.index.year <2018)][[predictor, "O3"]].asfreq('H')
    #else:
    #    #df = ozone[ (ozone.index.year > 1997) & (ozone.index.year <2018)][[predictor, "O3"]].asfreq('H')
    #    df = ozone[ (ozone.index.year > 1997) & (ozone.index.year <2018)][[predictor, "O3"]]
    
    
    
    TITLE = 'Corr for {}hrs - {}. \nBetween {}-11-01 and {}-03-31:'
    TITLE=TITLE.format(maxhour,dataset_name, INITIAL_YEAR, LAST_YEAR)
    print(TITLE)
    
    corr2={}
    global GG
    GG = df

    progress.max = (maxhour+1)*(1+calc_MI.value) #len(group)*maxhour
    progress.value = 0
    progress.description = 'Loading:'
    display(progress)
    progress.layout.visibility = 'visible'
    
    pd2 = pd.DataFrame()
    for filter_hour in range(filter_hour,filter_hour+1):
        progress. description= 'Loading: {}H'.format(filter_hour)
        corr = []
        mi = []
        if season == 'Invierno':
                df_filter = (df.index.month>=5) & (df.index.month <= 8)
        else:
                df_filter = (df.index.month>=11) | (df.index.month <= 3)

        df_filter = (df_filter) & (df.index.hour==filter_hour)

        initial_hour = filter_hour
        if start_lag0H.value == True:
            start_range = 0
        else:
            df[predictor] = df[predictor].shift(initial_hour)
            start_range = initial_hour+1
        
        for delta in range(start_range, maxhour+1):#range(0,maxhour+1):
        #for delta in range(0,maxhour+1):
            progress.value += 1
            if delta != 0:
                df[predictor] = df[predictor].shift(1)

            corr.append( df[ df_filter ].corr()[predictor][1] ) # Correlacion
            if calc_MI.value:
                tmpdf = df[ df_filter].dropna()
                #print( mutual_info_regression(tmpdf[predictor].values.reshape(-1, 1)[:5], tmpdf["O3"].values[:5] )[0] )
                #print("delta:",delta)
                if not tmpdf[predictor].empty:
                    tmp_mi =[]
                    for seed in [123,456,2,56,8,23,8978,11,500,92]:
                        tmp_mi.append( mutual_info_regression(tmpdf[predictor].values.reshape(-1, 1), tmpdf["O3"].values, random_state=seed )[0] )
                    mi.append( np.mean(tmp_mi))
                else:
                    mi.append( np.nan )
                progress.value += 1
                
        #corr2['Corr {}H'.format(filter_hour)] = corr

        
        data = {'Delta': range(start_range, maxhour+1),
                'Corr {}H'.format(filter_hour): corr}
        if calc_MI.value:
            data['MI {}H'.format(filter_hour)]= mi
            #print(mi)
        temp = pd.DataFrame(data)
        temp = temp.set_index('Delta')
        pd2 = pd.concat([pd2, temp], axis=1)
    if calc_MI.value:        
        vlines = [ pd2['MI {}H'.format(filter_hour)].idxmax()]
    else:
        vlines = [ pd2['Corr {}H'.format(filter_hour)].apply(np.abs).idxmax()]
    pd2.iplot(vline=vlines)
    #data = {'Delta': range(0,maxhour+1),
    #        'Correlation':corr
    #       }
    #corrdf = pd.DataFrame(data)
    #corrdf = corrdf.set_index('Delta')
    #progress.layout.visibility = 'hidden'
    #corrdf.iplot(title=title, vline=[ 1*12*24, 2*12*24])#, hline=[-1,1])
    #print("maximo", corrdf.idxmax(),corrdf.max())
    #print("minimo", corrdf.idxmin(),corrdf.min())

#general_layout = Layout(min_width="10em", max_width="15em")
general_layout = Layout(min_width="10px", max_width="200px")

seasons = widgets.Dropdown(options=['Invierno', 'Verano'],
                          value='Verano',
                            description='Season',
                            layout = general_layout
                          )
predictors = widgets.Dropdown(options=list(ozone.columns),
                             value='UVA',
                              description='Predictor:',
                              layout = general_layout
                             )
maxhours = widgets.BoundedIntText(value=24*7*4,min=0,max=24*7*4, description=' ',layout = general_layout)
maxhoursSlider = widgets.IntSlider(value=24*7*4,min=0,max=24*7*4, description='Lags in hours')
hour = widgets.BoundedIntText(value=13,min=1,max=23, description=' ',layout = general_layout)
hourSlider = widgets.IntSlider(value=13,min=1,max=23, description='Hour')
if COLAB:
    maxhoursSlider.disabled = True
    hourSlider.disabled = True
    maxhoursSlider.layout.visibility= "hidden"
    maxhours.description = "Lags in hours"
    hourSlider.layout.visibility= "hidden"
    hour.description = "Hour"

iseason = widgets.IntText(value=2004, description='initial season',layout = general_layout)
fseason = widgets.IntText(value=2013, description='final season',layout = general_layout)
iseason.value = 2004
fseason.value = 2013
#iseason.disabled = True
#fseason.disabled = True
calc_MI = widgets.Checkbox(value=False, description="MI")
start_lag0H = widgets.Checkbox(value=True, description="Start Lag from 0H")
l = widgets.jslink((maxhours, 'value'), (maxhoursSlider, 'value'))
l2 = widgets.jslink((hour, 'value'), (hourSlider, 'value'))
button = widgets.Button(description='Plot')
button.on_click(button_on_click)


w = interactive(plot_corr)
wgslider = []
output = w.children[0]

seasonVBox = widgets.VBox([seasons, iseason, fseason])
midVBox = widgets.VBox([predictors, calc_MI])
sliderVBox = widgets.VBox([maxhoursSlider, maxhours,start_lag0H, hourSlider, hour])
wgs = [seasonVBox, midVBox, sliderVBox]
hbox = widgets.HBox(wgs)
mainvbox = widgets.VBox([hbox, button, output]) #progress, output])



display(mainvbox)

## Correlación y MI con O3 - Station: Las Condes.

* Initial Season de 0 es lo mismo que 1997

* Final Season de 0 es lo mismo que 2017

* Seleccionar MI para calcular Mutial Information en lugar de la correlación  



# 5) Busqueda de máxima correlación y MI.

Busca la máxima correlación y MI para una una determinada hora, en este caso está fijo para las *13 Horas* que es donde generalmente ocurren los máximos diarios de Ozono.

In [ ]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}
configure_plotly_browser_state()

#MAX_HOURS =  672
#INITIAL_SEASSON = 2009 #1997
#LAST_SEASSON = 2016 #2018


#INITIAL_YEAR = INITIAL_SEASSON
#LAST_YEAR = LAST_SEASSON + 1

#TITLE = '# Ranking of Correlation for {}hrs - {}. \n# Between {}-11-01 and {}-03-31:'
#display(Markdown(TITLE.format(MAX_HOURS,dataset_name, INITIAL_YEAR, LAST_YEAR)))
#display(Markdown("# Ranking for {}".format(dataset_name)))


from ipywidgets import widgets, Layout
from IPython.display import clear_output

skip = True


progress = widgets.IntProgress(
       value=0,
       min=0,
       max=100,
       step=1,
       description='Loading:',
       bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
       orientation='horizontal'
)


def corr(season, predictor, maxhour, FILTER_HOUR):
    MAX_HOURS =  rank_maxhours.value
    INITIAL_SEASSON = rank_iseason.value #1997
    LAST_SEASSON = rank_fseason.value #2018
    FILTER_HOUR = FILTER_HOUR
    START_LAG0H = True
    
    INITIAL_YEAR = INITIAL_SEASSON
    LAST_YEAR = LAST_SEASSON + 1
    
    
    filter_hour = FILTER_HOUR
    
    df = ozone[ (ozone.index.year >= INITIAL_YEAR) & (ozone.index.year <= LAST_YEAR)][[predictor, "O3"]]
    
    if predictor == 'O3':
        predictor = 'O3_2'
        df.columns = ["O3_2", "O3"]
        
    first_in_predictor = df[predictor].first_valid_index()
    if first_in_predictor == None:
        print('Max correlation for {} at delta {} hours: {}'.format(predictor, np.nan, np.nan))
        print('Max MI for {} at delta {} hours: {}'.format(predictor, np.nan, np.nan))
        return (np.nan, np.nan, np.nan), (np.nan, np.nan, np.nan)
    
    
    
    i = max( first_in_predictor, df['O3'].first_valid_index())
    f = df['O3'].last_valid_index()
    df = df[i:f]
    df = df.asfreq(freq='H')
    
    
    
    title = "Correlation for {} in {}".format(predictor, season)
    
    corr2={}

    progress.max = (maxhour+1)*(2) #len(group)*maxhour
    progress.value = 0
    progress.description = predictor+":"
    progress.layout.visibility = 'visible'
    
    pd2 = pd.DataFrame()
    for filter_hour in range(filter_hour,filter_hour+1):
        corr_list = []
        mi_list = []
        if season == 'Invierno':
                df_filter = (df.index.month>=5) & (df.index.month <= 8)
        else:
                df_filter = (df.index.month>=11) | (df.index.month <= 3)

        df_filter = (df_filter) & (df.index.hour==filter_hour)

        initial_hour = filter_hour
        if START_LAG0H == True:
            start_range = 0
        else:
            df[predictor] = df[predictor].shift(initial_hour)
            start_range = initial_hour+1
        
        for delta in range(start_range, maxhour+1):#range(0,maxhour+1):
        #for delta in range(0,maxhour+1):
            #print(len(df))
            progress.value += 1
            if delta != 0:
                df[predictor] = df[predictor].shift(1)

            corr_list.append( df[ df_filter ].corr()[predictor][1] ) # Correlacion
            
            tmpdf = df[ df_filter].dropna()
            #print( mutual_info_regression(tmpdf[predictor].values.reshape(-1, 1)[:5], tmpdf["O3"].values[:5] )[0] )
            #print("delta:",delta)
            if not tmpdf[predictor].empty:
                tmp_mi =[]
                for seed in [123,456,2,56,8,23,8978,11,500,92]:
                    tmp_mi.append( mutual_info_regression(tmpdf[predictor].values.reshape(-1, 1), tmpdf["O3"].values, random_state=seed )[0] )
                mi_list.append( np.mean(tmp_mi))
            else:
                mi_list.append( np.nan )
            progress.value += 1
            
        #corr2['Corr {}H'.format(filter_hour)] = corr
    
    progress.layout.visibility = 'hidden'
    
    absolute_corr = max(map(lambda x: (abs(x), x, corr_list.index(x)), corr_list))
    absolute_mi = max(map(lambda x: (abs(x), x, mi_list.index(x)), mi_list))
    
    ab_corr, value_corr, i_corr = absolute_corr
    ab_mi, value_mi, i_mi = absolute_mi
    print('Max correlation for {} at delta {} hours: {}'.format(predictor, i_corr, value_corr))
    print('Max MI for {} at delta {} hours: {}'.format(predictor, i_mi, value_mi))
    return (absolute_corr, absolute_mi)#(ab_corr, value_corr, i_corr)


def button_on_click(b):
    clear_output()
    display(mainHBox)

    MAX_HOURS =  rank_maxhours.value
    FILTER_HOUR = 13
    INITIAL_SEASSON = rank_iseason.value #1997
    LAST_SEASSON = rank_fseason.value #2018
    
    INITIAL_YEAR = INITIAL_SEASSON
    LAST_YEAR = LAST_SEASSON + 1

    FILE_STORE = file_widget.value

    TITLE = '# Ranking for {} hrs. max lags \n## at {} Hours - {}. \n## Sólo Verano \n## Between {}-11-01 and {}-03-31:'
    display(Markdown(TITLE.format(MAX_HOURS, FILTER_HOUR, dataset_name, INITIAL_YEAR, LAST_YEAR)))
    #display((TITLE.format(MAX_HOURS,dataset_name, INITIAL_YEAR, LAST_YEAR)))
    
    b.disabled = True
    progress.layout.visibility='hidden'
    display(progress)

    predictores_texto = " " + " ".join(list(ozone.columns)) + " "
    label_predictores = widgets.Label(value = predictores_texto)
    display(label_predictores)
    
    all_corr = {'Invierno':[], 'Verano':[]}
    all_mi = {'Invierno':[], 'Verano':[]}
    
    for season in ["Verano"]:#['Invierno', 'Verano']:
        print(season)
        for predictor in list(ozone.columns):
            #ab, m, i = corr(season, predictor, MAX_HOURS)
            actual_pred_text = predictores_texto.replace(predictor, "||"+predictor+"||")
            label_predictores.value = actual_pred_text
            corr_value, mi_value = corr(season, predictor, MAX_HOURS, FILTER_HOUR)
            ab, m, i = corr_value
            if np.isnan(ab):
                dato = (float('-inf'), m, predictor, i)
            else:
                dato = (ab, m, predictor, i)
            all_corr[season].append( dato )
            
            ab, m, i = mi_value
            if np.isnan(ab):
                dato = (float('-inf'), m, predictor, i)
            else:
                dato = (ab, m, predictor, i)
            all_mi[season].append( dato )
    
    print("\n"*3)
    print("-----------------------")
    #display(Markdown("## Ordered Correlations {} Hrs. max lags - {}. Between {}-11-01 and {}-03-31".format(MAX_HOURS, dataset_name, INITIAL_YEAR, LAST_YEAR)))
    
    '''
    for k in all_corr:
        all_corr[k].sort()
        all_corr[k].reverse()
    
    for k in all_mi:
        all_mi[k].sort()
        all_mi[k].reverse()
    '''
    
    for season in ['Verano']:#['Invierno', 'Verano']:
        #print("\n"+season+"  "+"------NOT SORTED------")
        if REWRITE.value == True:
            file_name = "{}_{}_{}_{}_{}H_corr_mi.csv"
            file_name = file_name.format(dataset_sigla, INITIAL_SEASSON, LAST_SEASSON,MAX_HOURS,FILTER_HOUR)
            path_file = "search_outputs/"+file_name
            #path_file = path_file.format(dataset_sigla, INITIAL_SEASSON, LAST_SEASSON,MAX_HOURS,FILTER_HOUR)
            
            file = open(path_file,"w")
            print("Escribiendo:",path_file)
            file.write("station season predictor correlation corr_lag corr_abs MI MI_lag\n")
            for corr_info, mi_info in zip(all_corr[season], all_mi[season]):
                corr_abs, correlation, predictor, corr_lag = corr_info
                __, mi, __test_pred, mi_lag = mi_info
                if __test_pred != predictor:
                    print("MAAAAAAAAAAAAAAAAAAL")
                print('\tMax correlation for {} at delta {} hours: {}  '.format(predictor, corr_lag, correlation))
                print('\tMax MI for {} at delta {} hours: {}  '.format(predictor, mi_lag, mi))
                if REWRITE.value == True:
                    file.write("{} {} {} {} {} {} {} {}\n".format(dataset_sigla,season, predictor, correlation,
                                                                    corr_lag, corr_abs, mi, mi_lag ))
        '''
        print("--")
        for ab, m, p, i in all_mi[season]:
            print('Max MI for {} at delta {} hours: {}  '.format(p, i, m))
            if REWRITE.value == True:
                file.write("{} {} {} {} {} {} {}\n".format(dataset_sigla, "MI",season, p,i,m,ab))
        '''
        if REWRITE.value == True:
            print("Fin de la escritura")
            file.close()
            if FILE_STORE == "Default":
                ALL_file_name = "{}_{}_{}_{}_{}H_corr_mi.csv"
                ALL_file_name = ALL_file_name.format("ALL", INITIAL_SEASSON, LAST_SEASSON,MAX_HOURS,FILTER_HOUR)
            else:
                ALL_file_name = FILE_STORE+".csv"
            all_open = open("stored_files/"+ALL_file_name, "a")
            all_open.write(file_name+"\n")
            all_open.close()
            print("{} agregado a {}".format(file_name, ALL_file_name))
    b.disabled = False


general_layout = Layout(min_width="10px", max_width="200px")
button = widgets.Button(description='Ranking')
REWRITE = widgets.Checkbox(value=False, description="(re)write file?")
rank_iseason = widgets.IntText(value=2004, description='Initial season', layout=general_layout)
rank_fseason = widgets.IntText(value=2017, description='Final season', layout=general_layout)
rank_maxhours = widgets.BoundedIntText(value=24*7*4,min=0,max=24*7*4, description='Lags', layout=general_layout)
file_widget = widgets.Text(value="Default", description="CSV to Store")
button.on_click(button_on_click)

leftVBox = widgets.VBox([button, REWRITE])
midVBox = widgets.VBox([rank_iseason, rank_fseason])
rightVBox = widgets.VBox([rank_maxhours, file_widget])
mainHBox = widgets.HBox([leftVBox,midVBox, rightVBox])
display(mainHBox)

# Ranking for 672 hrs. max lags 
## at 13 Hours - Cerrillos. 
## Sólo Verano 
## Between 2004-11-01 and 2018-03-31:

IntProgress(value=0, bar_style='success', description='Loading:', layout=Layout(visibility='hidden'))

Label(value=' CH4 CO PM10 PM25 NO2 NO NOX SO2 WD RH TEMP WS HCNM UVA UVB O3 WD-sin WD-cos ')

Verano
Max correlation for CH4 at delta 1 hours: 0.44425897080371196
Max MI for CH4 at delta 641 hours: 0.27265446227370643
Max correlation for CO at delta 7 hours: 0.4827849470647346
Max MI for CO at delta 7 hours: 0.3287127934768381
Max correlation for PM10 at delta 8 hours: 0.5265959286673065
Max MI for PM10 at delta 8 hours: 0.18325136004322803
Max correlation for PM25 at delta 6 hours: 0.5307749404951219
Max MI for PM25 at delta 6 hours: 0.18214659287828033
Max correlation for NO2 at delta 9 hours: 0.5960625238604791
Max MI for NO2 at delta 9 hours: 0.29215055244260396
Max correlation for NO at delta 7 hours: 0.47662953803889974
Max MI for NO at delta 9 hours: 0.146061349240911
Max correlation for NOX at delta 8 hours: 0.49974412795422063
Max MI for NOX at delta 9 hours: 0.2008478754836005
Max correlation for SO2 at delta 0 hours: 0.3748434199022984
Max MI for SO2 at delta 0 hours: 0.1228041904808265
Max correlation for WD at delta 7 hours: -0.3008039236443329
Max MI for WD at del

# 6) Gráfico de máximas correlaciones y MI
Grafica la información encontrada y escrita en los archivos de la celda de busqueda.

In [11]:
#@title Doble-click para ver/ocultar el código {display-mode: "form"}
configure_plotly_browser_state()
display(Markdown("# Gráfico de máximas correlaciones y MI - Todas las estaciones."))
display(Markdown("Aveces no aparece el gráfico o se duplica con alguno anterior. Presionar el botón refrescar hasta que aparezca."))

from IPython.display import display, clear_output
from ipywidgets import Layout

import os
import pandas as pd

files_path = "./stored_files/"
FILES = [f for f in os.listdir(files_path) if os.path.isfile(os.path.join(files_path, f))]
#print(FILES)

COLORS = ["#1f77b4",
         "#ff7f0e",
         "#2ca02c",
         "#dd2755",#"#d62728",
         "#ff00ff",#"#9467bd",
         "#8c564b",
         "#e377c2",
         "#7f7f7f",
         "#bcbd22",
         "#17becf",
         "#000000",#"#1f77b4",
         "#0000ff",#"#ff9f0e",
         "#ff0000",#"#2ca05c",
         "#bbbbbb"]#"#d62708"]
STATIONS = ["None"] #, "CONDES", "INDEP", "POH", "CERRI"]
#df_load = pd.read_csv("stored_files/all_corr_mi.csv", sep=" ").replace(float("-inf"), np.nan )


def show_plot(File, Type="correlation", SortBy='None', plot="Max"):
    #Type = typewg.value
    #SortBy = sortbywg.value
    BANNED_COLS = ["HCNM", "NO2", "SO2", "CH4"]
    df_temp = pd.DataFrame()
    with open(files_path+File) as file:
        for line in file:
            line = line.strip()
            #print(line)
            if df_temp.empty:
                df_temp = pd.read_csv("search_outputs/"+line, sep=" ").replace(float("-inf"), np.nan )
            else:
                df_temp2 = pd.read_csv("search_outputs/"+line, sep=" ").replace(float("-inf"), np.nan )
                df_temp = pd.concat([df_temp, df_temp2])
    STATIONS = ["None"] + list(df_temp["station"].unique())
    FIRST_STATION = STATIONS[1]
    LAST_STATION = STATIONS[-1]
    sortbywg.options= STATIONS


    #df_load = pd.read_csv("stored_files/"+File, sep=" ").replace(float("-inf"), np.nan )
    df_load = df_temp.copy()
    df = df_load.copy()
    COLS = df.pivot(index = "station", columns="predictor", values=Type).drop(BANNED_COLS, axis=1).columns
    dictCOLORS = { COLS[i]:COLORS[i] for i in range(len(COLS))}
    
    
    if Type == "correlation":
        values = ["correlation", "corr_abs"]
    elif Type == "MI":
        values = "MI"
    elif Type == "BOTH":
        dd = df.copy()
        scaler = MinMaxScaler()
        station = SortBy
        if station == "None":
            station = "CONDES"
        scaler.fit(dd[dd["station"]==station]["MI"].values.reshape(-1,1))
        df["MI"] = scaler.transform(dd["MI"].values.reshape(-1,1))
        scaler = MinMaxScaler()
        scaler.fit(dd[dd["station"]==station]["correlation"].apply(np.abs).values.reshape(-1,1))
        df["correlation"] = scaler.transform(dd["correlation"].apply(np.abs).values.reshape(-1,1))
        values = ["correlation", "MI"]
        BANNED_COLS2 = []
        for i in BANNED_COLS:
            for b in values:
                BANNED_COLS2.append( (i,b) )
        BANNED_COLS = BANNED_COLS2
    
    if plot == "Max":
        # Maxi
        show_df = df.pivot(index = "station", columns="predictor", values=values)
        #show_df = df.pivot(index = "station", columns="predictor", values="MI")
        show_df = show_df.reindex(STATIONS)
    
        if Type == "correlation":
            t1 = show_df[Type]
            t2 = show_df["corr_abs"]
            t2.index = map(lambda x: x+"_abs", t2.index)
            show_df = pd.concat([t1,t2])
            if SortBy != "None":
                show_df = show_df.sort_values(by = SortBy+'_abs', axis=1, ascending=False)
        else:
            #show_df = show_df[Type]
            if SortBy != "None":
                show_df = show_df.sort_values(by = SortBy, axis=1, ascending=False)
        if Type == "BOTH":
            show_df = show_df.swaplevel(axis=1)#.iplot(kind="bar")
            show_df = show_df.sort_index(axis=1, sort_remaining=False)
        show_df = show_df[FIRST_STATION : LAST_STATION].drop(BANNED_COLS, axis=1)
        #display(show_df["O3"])
        #output.clear_output()
        #configure_plotly_browser_state()
        show_df.iplot(kind="bar", colors=dictCOLORS)
    else:
        # MaxiHour
        if Type == "correlation":
            values = "corr_lag"
        elif Type == "MI":
            values = "MI_lag"
        if Type != "BOTH":
            show_df = df.pivot(index = "station", columns="predictor", values=values)
            show_df = show_df.reindex(STATIONS)
            if SortBy != "None":
                show_df = show_df.sort_values(by =SortBy, axis=1, ascending=True)
            show_df = show_df[FIRST_STATION : LAST_STATION].drop(BANNED_COLS, axis=1)
            #configure_plotly_browser_state()
            show_df.iplot(kind="bar", colors=dictCOLORS)
    refresh_button.disabled = False
    

general_layout = Layout(min_width="10px", max_width="200px")


filewg = widgets.Dropdown(options=FILES,
                          description="File",
                         )
typewg = widgets.Dropdown(options=["correlation","MI"],#,"BOTH"],
                          value='correlation',
                          description='Type',
                          layout = general_layout,
                          )
sortbywg = widgets.Dropdown(options=STATIONS,
                          value='None',
                          description='Sort By',
                          layout = general_layout,
                          )
plotwg = widgets.Dropdown(options=["Max", "Hour of Max"],
                          value='Max',
                          description='Plot',
                          layout = general_layout,
                          )
w = interactive(show_plot, File=filewg, Type=typewg, SortBy=sortbywg, plot=plotwg)

refresh_button = widgets.Button(description="Refresh")
def refresh_function(b):
    b.disabled = True
    w.update()

refresh_button.on_click(refresh_function)

HBox = widgets.HBox([typewg, sortbywg, plotwg])
output = w.children[-1]
mainBox = widgets.VBox([refresh_button, filewg,HBox, output])
display( mainBox)
w.update()

# Gráfico de máximas correlaciones y MI - Todas las estaciones.

Aveces no aparece el gráfico o se duplica con alguno anterior. Presionar el botón refrescar hasta que aparezca.

# Comentarios:
Hice unos arreglos en el cálculo de la correlación.
* Es más eficiente, ya no tarda tanto.
* Y ahora, se consideran horas que normalmenten no pertenecen a determinada estación (invierno o verano), pero que al aplicar el delta de horas, pasan a estar tendro del rango. Por ejemplo:
 * Antes: Para el 1 de Noviembre en verano, un delta de 24 horas en algún predictor, **dejaría puros datos NaN** para el 1 de Noviembre, ya que Octubre no se consideraba dentro del verano.
 * Ahora: Para el 1 de Noviembre en verano, un delta de 24 horas en algún predictor, si incluirá datos para calcular la correlación el 1 de Noviembre, ya que se considera la información de meses anteriores, pero el rango de datos para la correlación sigue siendo entre Noviembre y Marzo (para Verano).  

Hay algunas correlaciones que me llaman la atención, las que son las más altas cuando el delta es muy grande. Me causa extrañesa que por ejemplo, para Independencia, que la mayor correlación entre el UVB y el O3 se dé con un delta de 408 horas, lo que yo había entendido de los químicos era que la radiación solar influye directamente en la generación del O3 del mismo día. Eso no deja de ser cierto, ya que la correlación en el instante delta 0, también es alta y similar a la más alta en delta 408, y al aumentar el delta se ve una función cíclica, pero me llama la atención que la mayor correlación sea varias horas antes (17 días), ¿Será sólo un tema casual dado por la periodicidad de la correlación? ¿o efectivamente habría una dependencia química o física entre el ozono y el UVB de hace 17 días, pero sólo en independencia? ¿o se podría deber a los tipos de datos de la estación de Independencia? ya que con ese predictor, la situación sólo se da en Independencia, pero para otros predictores se da en otras estaciones también, por ejemplo para el UVA se da algo similar en todas las estaciones.

# OLD

Códigos anteriores se mantienen en caso de necesitar referencia.

## Correlation and MI (OLD)
Código base inicial para hacer los cálculos. Podría tener diferencias con los métodos anteriores.

Se mantiene sólo por si acaso.

In [ ]:
configure_plotly_browser_state()
#code_toggle()
MAX_HOURS = 672
INITIAL_SEASSON = 2009 #1997
LAST_SEASSON = 2016 #2018

INITIAL_YEAR = INITIAL_SEASSON
LAST_YEAR = LAST_SEASSON + 1

TITLE = '# Ranking of Correlation for {}hrs - {}. \n# Between {}-11-01 and {}-03-31:'
#display(Markdown(TITLE.format(MAX_HOURS,dataset_name, INITIAL_YEAR, LAST_YEAR)))
display((TITLE.format(MAX_HOURS,dataset_name, INITIAL_YEAR, LAST_YEAR)))


from ipywidgets import widgets

skip = True


progress = widgets.IntProgress(
       value=0,
       min=0,
       max=100,
       step=1,
       description='Loading:',
       bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
       orientation='horizontal'
)


def corr(season, predictor, maxhour):
    #if season == 'Invierno':
    #    group = gpi_hourly
    #else:
    #    group = gpv_hourly
    df = ozone[ (ozone.index.year >= INITIAL_YEAR) & (ozone.index.year <=LAST_YEAR)][[predictor, "O3"]]#.asfreq(freq='H')
    
    first_in_predictor = df[predictor].first_valid_index()
    if first_in_predictor == None:
        print('Max correlation for {} at delta {} hours: {}'.format(predictor, 0, np.nan))
        return (np.nan, np.nan, 0)
        
    i = max( df[predictor].first_valid_index(), df['O3'].first_valid_index())
    
    f = df['O3'].last_valid_index()
    df = df[i:f]
    df = df.asfreq(freq='H')
    
    '''
    if season == 'Invierno':
        #group = gpi_hourly
        #df = invierno.asfreq(freq='H')[[predictor, "O3"]]
        df = ozone[ (ozone.index.year > 1997) & (ozone.index.year <2018)].asfreq(freq='H')[[predictor, "O3"]]
    else:
        #group = gpv_hourly
        #df = verano.asfreq(freq='H')[[predictor, "O3"]]
        df = ozone[ (ozone.index.year > 1997) & (ozone.index.year <2018)].asfreq(freq='H')[[predictor, "O3"]]
    '''
    
    if predictor == 'O3':
        predictor = 'O3'
        df.columns = ["O3", "O3_2"]
        
    title = "Correlation for {} in {}".format(predictor, season)
    corr_list = []
    
    progress.max = maxhour+1#len(group)*maxhour
    progress.value = 0
    progress.description = predictor+":"
    progress.layout.visibility = 'visible'
    
    if season == 'Invierno':
        df_filter = (df.index.month>=5) & (df.index.month <= 8)
    else:
        df_filter = (df.index.month>=11) | (df.index.month <= 3)
    
    for delta in range(0,maxhour+1):
        '''
        df = pd.DataFrame()
        for year, gdf in group:
            dftemp = gdf[[predictor, 'O3']]
            
            ##dftemp = utils.o3delta(dftemp, delta)
            kk = dftemp.copy()
            kk[predictor] = kk[predictor].shift(delta)
            df = pd.concat([df, kk])
            ##df = pd.concat([df, dftemp])
        '''
        progress.value += 1
        if delta != 0:
            df[predictor] = df[predictor].shift(1)

        corr_list.append( df[df_filter].corr()[predictor][1] )
    
    progress.layout.visibility = 'hidden'
    
    absolute = max(map(lambda x: (abs(x), x, corr_list.index(x)), corr_list))
    
    ab, value, i = absolute
    print('Max correlation for {} at delta {} hours: {}'.format(predictor, i, value))
    return (ab, value, i)


def button_on_click(b):
    b.disabled = True
    progress.layout.visibility='hidden'
    display(progress)
    
    all = {'Invierno':[], 'Verano':[]}
    
    for season in ['Invierno', 'Verano']:
        print(season)
        for predictor in list(ozone.columns):
            ab, m, i = corr(season, predictor, MAX_HOURS)
            if np.isnan(ab):
                dato = (float('-inf'), m, predictor, i)
            else:
                dato = (ab, m, predictor, i)
            all[season].append( dato )
    
    print("\n"*3)
    print("-----------------------")
    print("## Ordered Correlations {} Hrs - {}. Between {}-11-01 and {}-03-31".format(MAX_HOURS, dataset_name, INITIAL_YEAR, LAST_YEAR))
    
    all['Invierno'].sort()
    all['Invierno'].reverse()
    all['Verano'].sort()
    all['Verano'].reverse()
    
    for season in ['Invierno', 'Verano']:
        print("\n"+season+"  ")
        for ab, m, p, i in all[season]:
            print('Max correlation for {} at delta {} hours: {}  '.format(p, i, m))
    b.disabled = False

button = widgets.Button(description='Ranking')
button.on_click(button_on_click)
display(button)

NameError: ignored

## Gráfico de máximas correlaciones y MI (OLD)
Grafica la información encontrada y escrita en los archivos de la celda anterior.

IMPORTANTE: Sólo lee 1 archivo, en ese archivo hay que unir los resultados de todas las estaciones en la misma categoría.

In [ ]:
configure_plotly_browser_state()
display(Markdown("# Gráfico de máximas correlaciones y MI - Todas las estaciones."))

from ipywidgets import Layout
import os

files_path = "./stored_files/"
FILES = [f for f in os.listdir(files_path) if os.path.isfile(os.path.join(files_path, f))]
#print(FILES)

COLORS = ["#1f77b4",
         "#ff7f0e",
         "#2ca02c",
         "#dd2755",#"#d62728",
         "#ff00ff",#"#9467bd",
         "#8c564b",
         "#e377c2",
         "#7f7f7f",
         "#bcbd22",
         "#17becf",
         "#000000",#"#1f77b4",
         "#0000ff",#"#ff9f0e",
         "#ff0000",#"#2ca05c",
         "#bbbbbb"]#"#d62708"]
STATIONS = ["None", "CONDES", "INDEP", "POH", "CERRI"]
#df_load = pd.read_csv("search_output/all_corr_mi.csv", sep=" ").replace(float("-inf"), np.nan )


def show_plot(File, Type="correlation", SortBy='None', plot="Max"):
    #Type = typewg.value
    #SortBy = sortbywg.value
    BANNED_COLS = ["HCNM", "NO2", "SO2", "CH4"]
    '''
    df_temp = None
    with open(files_path+File) as file:
        for line in file:
            line = line.strip()
            #print(line)
            #Hacer dataframe con cada linea del archivo
    '''

    #df_load = pd.read_csv("search_output/"+File, sep=" ").replace(float("-inf"), np.nan )
    df_load = pd.read_csv("search_output/all_corr_mi_old.csv", sep=" ").replace(float("-inf"), np.nan )
    df = df_load.copy()
    COLS = df.pivot(index = "station", columns="predictor", values=Type).drop(BANNED_COLS, axis=1).columns
    dictCOLORS = { COLS[i]:COLORS[i] for i in range(len(COLS))}
    
    if Type == "correlation":
        values = ["correlation", "corr_abs"]
    elif Type == "MI":
        values = "MI"
    elif Type == "BOTH":
        dd = df.copy()
        scaler = MinMaxScaler()
        station = SortBy
        if station == "None":
            station = "CONDES"
        scaler.fit(dd[dd["station"]==station]["MI"].values.reshape(-1,1))
        df["MI"] = scaler.transform(dd["MI"].values.reshape(-1,1))
        scaler = MinMaxScaler()
        scaler.fit(dd[dd["station"]==station]["correlation"].apply(np.abs).values.reshape(-1,1))
        df["correlation"] = scaler.transform(dd["correlation"].apply(np.abs).values.reshape(-1,1))
        values = ["correlation", "MI"]
        BANNED_COLS2 = []
        for i in BANNED_COLS:
            for b in values:
                BANNED_COLS2.append( (i,b) )
        BANNED_COLS = BANNED_COLS2
    
    if plot == "Max":
        # Maxi
        show_df = df.pivot(index = "station", columns="predictor", values=values)
        #show_df = df.pivot(index = "station", columns="predictor", values="MI")
        show_df = show_df.reindex(STATIONS)
    
        if Type == "correlation":
            t1 = show_df[Type]
            t2 = show_df["corr_abs"]
            t2.index = map(lambda x: x+"_abs", t2.index)
            show_df = pd.concat([t1,t2])
            if SortBy != "None":
                show_df = show_df.sort_values(by = SortBy+'_abs', axis=1, ascending=False)
        else:
            #show_df = show_df[Type]
            if SortBy != "None":
                show_df = show_df.sort_values(by = SortBy, axis=1, ascending=False)
        if Type == "BOTH":
            show_df = show_df.swaplevel(axis=1)#.iplot(kind="bar")
            show_df = show_df.sort_index(axis=1, sort_remaining=False)
        show_df = show_df["CONDES" : "CERRI"].drop(BANNED_COLS, axis=1)
        show_df.iplot(kind="bar", colors=dictCOLORS)
    else:
        # MaxiHour
        if Type == "correlation":
            values = "corr_lag"
        elif Type == "MI":
            values = "MI_lag"
        if Type != "BOTH":
            show_df = df.pivot(index = "station", columns="predictor", values=values)
            show_df = show_df.reindex(STATIONS)
            if SortBy != "None":
                show_df = show_df.sort_values(by =SortBy, axis=1, ascending=True)
            show_df = show_df["CONDES" : "CERRI"].drop(BANNED_COLS, axis=1)
            show_df.iplot(kind="bar", colors=dictCOLORS)

general_layout = Layout(min_width="10px", max_width="200px")


filewg = widgets.Dropdown(options=FILES,
                          value="all_corr_mi.csv",
                          description="File",
                         )
typewg = widgets.Dropdown(options=["correlation","MI"],#,"BOTH"],
                          value='correlation',
                          description='Type',
                          layout = general_layout,
                          )
sortbywg = widgets.Dropdown(options=STATIONS,
                          value='None',
                          description='Sort By',
                          layout = general_layout,
                          )
plotwg = widgets.Dropdown(options=["Max", "Hour of Max"],
                          value='Max',
                          description='Plot',
                          layout = general_layout,
                          )
w = interactive(show_plot, File=filewg, Type=typewg, SortBy=sortbywg, plot=plotwg)


HBox = widgets.HBox([typewg, sortbywg, plotwg])
output = w.children[-1]
mainBox = widgets.VBox([filewg,HBox, output])
display( mainBox)
w.update()

# Gráfico de máximas correlaciones y MI - Todas las estaciones.

# Usando seno y coseno (WD)

## Ordered Correlations 672 Hrs - Las Condes. Between 2004-11-01 and 2014-03-31

Invierno  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for UVB at delta 2 hours: 0.689461858301072  
Max correlation for TEMP at delta 0 hours: 0.5477807662458793  
Max correlation for RH at delta 0 hours: -0.4790377784464738  
Max correlation for WD-sin at delta 0 hours: -0.4785251389992158  
Max correlation for WD at delta 0 hours: 0.4193267859464547  
Max correlation for HCNM at delta 0 hours: -0.3442708343748825  
Max correlation for NO at delta 0 hours: -0.3405754085685626  
Max correlation for PM10 at delta 9 hours: -0.3284781497411559  
Max correlation for NOX at delta 10 hours: -0.32423128273147106  
Max correlation for CO at delta 9 hours: -0.30309590282300247  
Max correlation for NO2 at delta 657 hours: -0.302353024812431  
Max correlation for PM25 at delta 9 hours: -0.30170484266684683  
Max correlation for WD-cos at delta 551 hours: -0.2834562507550269  
Max correlation for SO2 at delta 10 hours: -0.2813760613790816  
Max correlation for WS at delta 1 hours: 0.24154185957169944  
Max correlation for UVA at delta 1 hours: 0.13610560829344218  
Max correlation for CH4 at delta 0 hours: nan  

Verano  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for TEMP at delta 0 hours: 0.8417142044374804  
Max correlation for RH at delta 0 hours: -0.7541808183203014  
Max correlation for UVB at delta 0 hours: 0.7401824288716049  
Max correlation for WD-sin at delta 12 hours: 0.7149007973604778  
Max correlation for WD at delta 12 hours: -0.6941343706703654  
Max correlation for WS at delta 0 hours: 0.6877327303917132  
Max correlation for WD-cos at delta 11 hours: 0.521327677624642  
Max correlation for PM10 at delta 348 hours: -0.47477845678038827  
Max correlation for UVA at delta 288 hours: 0.45299467444574143  
Max correlation for PM25 at delta 3 hours: 0.42755968849449666  
Max correlation for SO2 at delta 0 hours: 0.4252164090358773  
Max correlation for NO at delta 198 hours: 0.41843457295827946  
Max correlation for HCNM at delta 552 hours: -0.4145308377586028  
Max correlation for NOX at delta 166 hours: -0.403119889803662  
Max correlation for CO at delta 309 hours: -0.3414370245603408  
Max correlation for NO2 at delta 334 hours: -0.32564573247929257  
Max correlation for CH4 at delta 0 hours: nan  

## Ordered Correlations 672 Hrs - Independencia. Between 2010-11-01 and 2018-03-31

Invierno  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for UVB at delta 1 hours: 0.7107455666249932  
Max correlation for TEMP at delta 0 hours: 0.503117836479027  
Max correlation for RH at delta 0 hours: -0.4869661240867894  
Max correlation for HCNM at delta 587 hours: 0.4277715032333099  
Max correlation for NO at delta 0 hours: -0.4001694467959127  
Max correlation for WS at delta 0 hours: 0.3756940780762238  
Max correlation for CO at delta 0 hours: -0.3696649983845782  
Max correlation for NOX at delta 0 hours: -0.3561601428301507  
Max correlation for WD-sin at delta 0 hours: -0.33037340443487806  
Max correlation for WD-cos at delta 20 hours: -0.27126926094408843  
Max correlation for PM25 at delta 0 hours: -0.2377223383621279  
Max correlation for WD at delta 0 hours: 0.23436672744590598  
Max correlation for PM10 at delta 659 hours: -0.21927636199563844  
Max correlation for NO2 at delta 658 hours: -0.21859550762058075  
Max correlation for SO2 at delta 658 hours: -0.19174237322523296  
Max correlation for UVA at delta 145 hours: 0.12059897785876794  

Verano  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for UVB at delta 408 hours: 0.8045985485117219  
Max correlation for TEMP at delta 0 hours: 0.79851139871688  
Max correlation for WS at delta 70 hours: 0.7374135228543902  
Max correlation for RH at delta 0 hours: -0.6503031873992928  
Max correlation for WD-cos at delta 9 hours: 0.6322960113076761  
Max correlation for NO at delta 559 hours: 0.481451744028487  
Max correlation for NOX at delta 6 hours: 0.4435854549992965  
Max correlation for PM10 at delta 4 hours: 0.39812959006065207  
Max correlation for NO2 at delta 166 hours: -0.3930812993582568  
Max correlation for CO at delta 6 hours: 0.37047574701848923  
Max correlation for PM25 at delta 5 hours: 0.3257345279955552  
Max correlation for WD-sin at delta 1 hours: -0.3087785460230516  
Max correlation for SO2 at delta 3 hours: 0.24720411606215917  
Max correlation for UVA at delta 648 hours: 0.23910820885982237  
Max correlation for WD at delta 50 hours: 0.17323596850574705  
Max correlation for HCNM at delta 0 hours: nan  

## Ordered Correlations 672 Hrs - Parque O'Higgins. Between 2009-11-01 and 2017-03-31

Invierno  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for UVB at delta 2 hours: 0.7787283241649763  
Max correlation for TEMP at delta 0 hours: 0.5780820460654323  
Max correlation for RH at delta 0 hours: -0.5532172782940635  
Max correlation for WS at delta 0 hours: 0.41912985607807846  
Max correlation for NO at delta 0 hours: -0.39373398565776013  
Max correlation for NOX at delta 0 hours: -0.3825543348651806  
Max correlation for CO at delta 0 hours: -0.3704550116015572  
Max correlation for HCNM at delta 0 hours: -0.3682438969233761  
Max correlation for WD-sin at delta 0 hours: -0.3140915524619891  
Max correlation for PM25 at delta 0 hours: -0.2799160610963501  
Max correlation for CH4 at delta 0 hours: -0.2711426696450948  
Max correlation for WD at delta 0 hours: 0.24415622168773193  
Max correlation for NO2 at delta 322 hours: -0.2373346083249748  
Max correlation for PM10 at delta 0 hours: -0.2160924447676669  
Max correlation for WD-cos at delta 453 hours: -0.16638246700911155  
Max correlation for UVA at delta 193 hours: 0.1265434690860384  
Max correlation for SO2 at delta 332 hours: -0.08315486683339184  

Verano  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for TEMP at delta 0 hours: 0.8199753097888861  
Max correlation for WS at delta 93 hours: 0.7560728141639702  
Max correlation for RH at delta 0 hours: -0.729423848002759  
Max correlation for UVB at delta 0 hours: 0.6582816564819054  
Max correlation for NOX at delta 7 hours: 0.5406146884003272  
Max correlation for NO at delta 7 hours: 0.5133497677820817  
Max correlation for NO2 at delta 166 hours: -0.5056527322252788  
Max correlation for PM25 at delta 5 hours: 0.4199391543466761  
Max correlation for CO at delta 7 hours: 0.39915092376984196  
Max correlation for PM10 at delta 5 hours: 0.390681801528893  
Max correlation for WD-cos at delta 20 hours: -0.3330780136079742  
Max correlation for SO2 at delta 3 hours: 0.2900193515036517  
Max correlation for UVA at delta 649 hours: 0.2598334744632426  
Max correlation for HCNM at delta 57 hours: 0.20763070301363634  
Max correlation for CH4 at delta 344 hours: 0.20732233838223377  
Max correlation for WD-sin at delta 0 hours: -0.11923745404517576  
Max correlation for WD at delta 1 hours: 0.08237801886481577  

## Ordered Correlations 672 Hrs - Cerrillos. Between 2004-11-01 and 2014-03-31

Invierno  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for UVB at delta 2 hours: 0.733096118442781  
Max correlation for RH at delta 0 hours: -0.5305215756521953  
Max correlation for TEMP at delta 0 hours: 0.5258857634528779  
Max correlation for WD-sin at delta 0 hours: -0.41005736575963414  
Max correlation for CO at delta 0 hours: -0.39435331640895516  
Max correlation for NO at delta 0 hours: -0.39275799021721475  
Max correlation for WS at delta 454 hours: 0.3924295010385974  
Max correlation for NOX at delta 0 hours: -0.3845660430888587  
Max correlation for WD at delta 1 hours: 0.33604830153502807  
Max correlation for CH4 at delta 295 hours: 0.33497072588394644  
Max correlation for PM25 at delta 0 hours: -0.30482670029086045  
Max correlation for WD-cos at delta 20 hours: -0.2931099283274601  
Max correlation for PM10 at delta 0 hours: -0.23703998845196353  
Max correlation for NO2 at delta 0 hours: -0.22606118875792475  
Max correlation for HCNM at delta 504 hours: 0.19841782793854335  
Max correlation for SO2 at delta 430 hours: -0.15917132597911185  
Max correlation for UVA at delta 17 hours: -0.1020462271593577  

Verano  
Max correlation for O3 at delta 0 hours: 1.0  
Max correlation for TEMP at delta 0 hours: 0.7787699824017785  
Max correlation for WS at delta 69 hours: 0.7578591942827497  
Max correlation for UVB at delta 0 hours: 0.6751050073446246  
Max correlation for RH at delta 0 hours: -0.6595061610640783  
Max correlation for NO2 at delta 142 hours: -0.5124087401018875  
Max correlation for NOX at delta 7 hours: 0.5065097426430784  
Max correlation for NO at delta 7 hours: 0.47276454503915455  
Max correlation for PM25 at delta 5 hours: 0.454207291993328  
Max correlation for UVA at delta 649 hours: 0.4209383065820623  
Max correlation for PM10 at delta 4 hours: 0.4064938962936237  
Max correlation for CH4 at delta 7 hours: 0.3883022162668325  
Max correlation for CO at delta 7 hours: 0.37647210850986385  
Max correlation for WD-cos at delta 6 hours: 0.3273375527053016  
Max correlation for SO2 at delta 4 hours: 0.3090563175176333  
Max correlation for WD-sin at delta 0 hours: -0.25471773183101615  
Max correlation for WD at delta 1 hours: 0.17238473246068217  
Max correlation for HCNM at delta 584 hours: 0.15238923550299072  

In [ ]:
# Restringido a años AÑOS
dCorr = {"CONDES":'''Max correlation for O3 at delta 0 hours: 1.0
Max correlation for TEMP at delta 0 hours: 0.8417142044374804
Max correlation for RH at delta 0 hours: -0.7541808183203014
Max correlation for UVB at delta 0 hours: 0.7401824288716049
Max correlation for WD-sin at delta 12 hours: 0.7149007973604778
Max correlation for WD at delta 12 hours: -0.6941343706703654
Max correlation for WS at delta 0 hours: 0.6877327303917132
Max correlation for WD-cos at delta 11 hours: 0.521327677624642
Max correlation for PM10 at delta 348 hours: -0.47477845678038827
Max correlation for UVA at delta 288 hours: 0.45299467444574143
Max correlation for PM25 at delta 3 hours: 0.42755968849449666
Max correlation for SO2 at delta 0 hours: 0.4252164090358773
Max correlation for NO at delta 198 hours: 0.41843457295827946
Max correlation for HCNM at delta 552 hours: -0.4145308377586028
Max correlation for NOX at delta 166 hours: -0.403119889803662
Max correlation for CO at delta 309 hours: -0.3414370245603408
Max correlation for NO2 at delta 334 hours: -0.32564573247929257
Max correlation for CH4 at delta 0 hours: nan ''',
        "INDEP":'''Max correlation for O3 at delta 0 hours: 1.0
Max correlation for UVB at delta 408 hours: 0.8045985485117219
Max correlation for TEMP at delta 0 hours: 0.79851139871688
Max correlation for WS at delta 70 hours: 0.7374135228543902
Max correlation for RH at delta 0 hours: -0.6503031873992928
Max correlation for WD-cos at delta 9 hours: 0.6322960113076761
Max correlation for NO at delta 559 hours: 0.481451744028487
Max correlation for NOX at delta 6 hours: 0.4435854549992965
Max correlation for PM10 at delta 4 hours: 0.39812959006065207
Max correlation for NO2 at delta 166 hours: -0.3930812993582568
Max correlation for CO at delta 6 hours: 0.37047574701848923
Max correlation for PM25 at delta 5 hours: 0.3257345279955552
Max correlation for WD-sin at delta 1 hours: -0.3087785460230516
Max correlation for SO2 at delta 3 hours: 0.24720411606215917
Max correlation for UVA at delta 648 hours: 0.23910820885982237
Max correlation for WD at delta 50 hours: 0.17323596850574705
Max correlation for HCNM at delta 0 hours: nan ''',
        "POH":'''Max correlation for O3 at delta 0 hours: 1.0
Max correlation for TEMP at delta 0 hours: 0.8199753097888861
Max correlation for WS at delta 93 hours: 0.7560728141639702
Max correlation for RH at delta 0 hours: -0.729423848002759
Max correlation for UVB at delta 0 hours: 0.6582816564819054
Max correlation for NOX at delta 7 hours: 0.5406146884003272
Max correlation for NO at delta 7 hours: 0.5133497677820817
Max correlation for NO2 at delta 166 hours: -0.5056527322252788
Max correlation for PM25 at delta 5 hours: 0.4199391543466761
Max correlation for CO at delta 7 hours: 0.39915092376984196
Max correlation for PM10 at delta 5 hours: 0.390681801528893
Max correlation for WD-cos at delta 20 hours: -0.3330780136079742
Max correlation for SO2 at delta 3 hours: 0.2900193515036517
Max correlation for UVA at delta 649 hours: 0.2598334744632426
Max correlation for HCNM at delta 57 hours: 0.20763070301363634
Max correlation for CH4 at delta 344 hours: 0.20732233838223377
Max correlation for WD-sin at delta 0 hours: -0.11923745404517576
Max correlation for WD at delta 1 hours: 0.08237801886481577 ''',
        "CERRI":'''Max correlation for O3 at delta 0 hours: 1.0
Max correlation for TEMP at delta 0 hours: 0.7787699824017785
Max correlation for WS at delta 69 hours: 0.7578591942827497
Max correlation for UVB at delta 0 hours: 0.6751050073446246
Max correlation for RH at delta 0 hours: -0.6595061610640783
Max correlation for NO2 at delta 142 hours: -0.5124087401018875
Max correlation for NOX at delta 7 hours: 0.5065097426430784
Max correlation for NO at delta 7 hours: 0.47276454503915455
Max correlation for PM25 at delta 5 hours: 0.454207291993328
Max correlation for UVA at delta 649 hours: 0.4209383065820623
Max correlation for PM10 at delta 4 hours: 0.4064938962936237
Max correlation for CH4 at delta 7 hours: 0.3883022162668325
Max correlation for CO at delta 7 hours: 0.37647210850986385
Max correlation for WD-cos at delta 6 hours: 0.3273375527053016
Max correlation for SO2 at delta 4 hours: 0.3090563175176333
Max correlation for WD-sin at delta 0 hours: -0.25471773183101615
Max correlation for WD at delta 1 hours: 0.17238473246068217
Max correlation for HCNM at delta 584 hours: 0.15238923550299072 '''}

In [ ]:
configure_plotly_browser_state()
d = { }
maxHour = {}
STATIONS = ['CONDES','INDEP','POH','CERRI']
for STATION in STATIONS:
    for l in dCorr[STATION].split("\n"):
        #print(l)
        l = l.split("for ")[1]
        pred = l.split(" at")[0]
        l = l.split("delta ")[1]
        hours = int(l.split(" hours")[0])
        value = float(l.split(": ")[1].strip())
        if pred not in d:
            d[pred] = []
        if pred not in maxHour:
            maxHour[pred] = []
        d[pred].append(value)
        if np.isnan(value):
            maxHour[pred].append( value)
        else:
            maxHour[pred].append( hours)
    if STATION == 'INDEP':
        d['CH4'].append(np.nan)
        maxHour['CH4'].append(np.nan)
    
df = pd.DataFrame(d)
df.index = STATIONS
df.iplot(kind='bar')#, hline=[-1,1])

maxHour
dfH = pd.DataFrame(maxHour)
dfH.index = STATIONS
dfH.iplot(kind='bar', hline=[672])

In [ ]:
configure_plotly_browser_state()
d = {"valor":range(0,361)}

df = pd.DataFrame(d)
#df['valor'] = df['valor'].apply(lambda x: (np.sin((x+45)*np.pi/180)+np.cos(((x+45)*np.pi/180))))
valor = df['valor'].values
prom = valor.mean()
std = valor.std()
mini = valor.min()
maxi = valor.max()
#df['valor'] = df['valor'].apply(lambda x: ((x-mini)/(maxi-mini)*2)-1)

df['sin'] = df['valor'].apply(lambda x: np.sin(x*np.pi/180))
df['cos'] = df['valor'].apply(lambda x: np.cos(x*np.pi/180))


df.iplot()